# Task 2 Encoding and Classifier

## Problem 
<br>
Encoding the following files in a quantum circuit mock_train_set.csv and mock_test_set.csv in at least two different ways (these could be basis, angle, amplitude, kernel or random encoding
<br>
<br>
● Design a variational quantum circuit for each of the encodings, uses the column 4 as the target, this is a binary class 0 and 1.<br>
● You must use the data from column0 to column3 for your proposed classifier.<br>
● Consider the ansatz you are going to design as a layer and find out how many layers are
necessary to reach the best performance. <br>

### Analyze and discuss the results

Feel free to use existing frameworks (e.g. PennyLane, Qiskit) for creating and training the circuits.<br><br>
This PennyLane demo can be useful: Training a quantum circuit with Pytorch,<br>
This Quantum Tensorflow tutorial can be useful: Training a quantum circuit with Tensorflow.<br>
For the variational circuit, you can try any circuit you want. You can start from one with a layer of RX, RZ and CNOTs.<br>
<br>
## References
* https://pennylane.ai/qml/demos/tutorial_state_preparation.html
* https://www.tensorflow.org/quantum/tutorials/mnist

<br>

## Clasification Using Amplitude Encoding on 2 Qubits

In [20]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from ipywidgets import widgets
from IPython.display import display, HTML

In [21]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

import sys
from math import sqrt, pi

import pandas as pd
import scipy
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
# supress a warning that is not useful here
pd.options.mode.chained_assignment = None

In [22]:
# sklearn StandardScaler
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

path_train = '/_jupyter/QC/QOSF-challenge-md-2022/task-02/mock_train_set.csv'
path_test = '/_jupyter/QC/QOSF-challenge-md-2022/task-02/mock_test_set.csv'

df = pd.read_csv(path_train)
df_c = df.copy(deep=True)
df['1'] = np.log10(df_c['1'])
df['2'] = np.log10(df_c['2'])

f = lambda x: -1.0 if x==0 else 1.0
df['4'] = df_c['4'].map(f)

# npdf = df2.to_numpy()
npdf = df.to_numpy()
data = np.array(npdf)

print("Train data standardised:\n", data)

X = data[:, 0:4]
Y = data[:, -1]

# scale the data using sklearn StandardScaler
# std_slc = StandardScaler(with_mean=False)
std_slc = StandardScaler(with_mean=True)
std_slc.fit(X)
X_std = std_slc.transform(X)

# normalize data using sklearn StandardScaler
normalizer = Normalizer().fit(X_std)  # fit does nothing.
X_norm = normalizer.transform(X_std)

# features will be the angles vector
features = np.array(X_norm, requires_grad=False)
print("Train data normalized:\n", features)




Train data standardised:
 [[ 2.78926e+03  3.00000e+00  1.00000e+00  2.00000e+01 -1.00000e+00]
 [ 4.04001e+03  6.00000e+00  0.00000e+00  1.00000e+00  1.00000e+00]
 [ 2.93120e+03  4.00000e+00  4.00000e+00  4.00000e+01  1.00000e+00]
 ...
 [ 4.18281e+03  0.00000e+00  0.00000e+00  6.50000e+01 -1.00000e+00]
 [ 3.11375e+03  4.00000e+00  2.00000e+00  1.00000e+00  1.00000e+00]
 [ 4.56757e+03  4.00000e+00  5.00000e+00  9.00000e+01  1.00000e+00]]


StandardScaler()

Train data normalized:
 [[ 0.08683356 -0.03797654 -0.66528081 -0.74055328]
 [ 0.37612267  0.45694046 -0.51293519 -0.62179952]
 [ 0.26356815  0.46027486  0.79974818 -0.28121474]
 ...
 [ 0.47981546 -0.57210908 -0.59413647  0.29911565]
 [ 0.1940783   0.21079313 -0.20305745 -0.93630526]
 [ 0.56534373  0.1530959   0.47275529  0.65836961]]


Circuit coding for angle encoding follows pennylane technique, whch is also following the scheme in in Schuld and Petruccione (2018).<br>
""We had to also decompose controlled Y-axis rotations into more basic circuits following Nielsen and Chuang (2010).""<br>

* https://link.springer.com/book/10.1007/978-3-319-96424-9
* 

In [23]:
num_qubits = 4
# num_layers = 2
num_layers = 8

dev = qml.device("default.gaussian", wires=num_qubits)

### test angle encoding

In [24]:
# trainable circuit layer
def layer(W):
    for i in range(num_qubits):

        mag_alpha =  W[i, 0]
        phase_alpha = W[i, 1]
        phi = W[i, 2]

        qml.Displacement(mag_alpha, phase_alpha, wires=i)
        qml.Rotation(phi, wires=i)

# circuit that gets evaluated and optimised on each step
@qml.qnode(dev)
def circuit(weights, data):

    qml.DisplacementEmbedding(features=data, wires=range(num_qubits), method='amplitude')

    # apply trainable layers
    for W in weights:
        layer(W)

    return qml.expval(qml.NumberOperator(wires=1))

draw_flag = 0
def variational_classifier(weights, bias, data):
    
    global draw_flag

    if draw_flag:
        draw_flag=0
        # qml.draw

    return circuit(weights, data) + bias

# ###############################################################################
# standard square loss
def square_loss(labels, predictions):
    
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

# ###############################################################################
# goal: maximize accuracy
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss


def cost(weights, bias, features, labels):
    
    predictions = [variational_classifier(weights, bias, f) for f in features]
    return square_loss(labels, predictions)


In [25]:
np.random.seed(0)
num_data = len(Y)

# num_train = int(0.75 * num_data)
num_train = int(0.75 * num_data)
index = np.random.permutation(range(num_data))

feats_train = features[index[:num_train]]
Y_train = Y[index[:num_train]]

feats_val = features[index[num_train:]]
Y_val = Y[index[num_train:]]

# We need these later for plotting
X_train = X[index[:num_train]]
X_val = X[index[num_train:]]

# ######################################################################
# Load the test dataset and apply same transformations as we did with the train dataset.
df_test = pd.read_csv(path_test)
df_test_c = df_test.copy(deep=True)
df_test['1'] = np.log10(df_test_c['1'])
df_test['2'] = np.log10(df_test_c['2'])

f = lambda x: -1.0 if x==0 else 1.0
df_test['4'] = df_test_c['4'].map(f)
data_test = df_test.to_numpy()

X_test_ini = data_test[:, 0:4]
Y_test = data_test[:, -1]

# scale data using sklearn StandardScaler
std_slc = StandardScaler(with_mean=False)
std_slc.fit(X_test_ini)
X_test_std = std_slc.transform(X_test_ini)

# normalize data using sklearn StandardScaler
normalizer = Normalizer().fit(X_test_std)  # fit does nothing.
X_test_norm = normalizer.transform(X_test_std)

# convert to a pennylane numpy array
X_test = np.array(X_test_norm, requires_grad=False)


# ######################################################################
# accuracy for test dtaset
def test_accuracy(weights, bias):
    # apply the variational clasifier circuit on test dataset
    # using the learned weights
    predictions_test = [np.sign(variational_classifier(weights, bias, f)) for f in X_test]

    return accuracy(Y_test, predictions_test)




StandardScaler(with_mean=False)

In [26]:
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

# initialise the optimizer
# opt = qml.GradientDescentOptimizer(stepsize=0.05)
opt = NesterovMomentumOptimizer(0.01)
# opt = NesterovMomentumOptimizer(0.008)
# opt = NesterovMomentumOptimizer(0.1)
# opt = NesterovMomentumOptimizer(0.1)
# batch_size = 5
# batch_size = 15
# batch_size = 20
batch_size = 10


# train the variational classifier
weights = weights_init
bias = bias_init

# steps = 50 # acc: 0.94
# steps = 149  # acc: 0.92
# steps = 150  # acc: 0.92
# steps = 42  # acc: 0.85

# phase
steps = 95 # acc:
# steps = 72 # acc:0.90


for it in range(steps):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, num_train, (batch_size,))
    feats_train_batch = feats_train[batch_index]
    Y_train_batch = Y_train[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, feats_train_batch, Y_train_batch)

    # Compute predictions on train and validation set
    predictions_train = [np.sign(variational_classifier(weights, bias, f)) for f in feats_train]
    predictions_val = [np.sign(variational_classifier(weights, bias, f)) for f in feats_val]

    # Compute accuracy on train and validation set
    acc_train = accuracy(Y_train, predictions_train)
    acc_val = accuracy(Y_val, predictions_val)

    acc_test = test_accuracy(weights, bias)

    # print(
    #     "Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} "
    #     "".format(it + 1, cost(weights, bias, features, Y), acc_train, acc_val)
    # )

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} | Acc test: {:0.7f} "
        "".format(it + 1, cost(weights, bias, features, Y), acc_train, acc_val, acc_test)
    )
    # if acc_train >= 0.88 and acc_val >= 0.88:
    if acc_train >= 0.95 and acc_val >= 0.95 and acc_test >= 0.95:
        # early stop
        break



Iter:     1 | Cost: 0.9744105 | Acc train: 0.5200000 | Acc validation: 0.4666667 | Acc test: 0.5166667 
Iter:     2 | Cost: 0.9158187 | Acc train: 0.6711111 | Acc validation: 0.6400000 | Acc test: 0.5166667 
Iter:     3 | Cost: 1.1297534 | Acc train: 0.7555556 | Acc validation: 0.7333333 | Acc test: 0.5166667 
Iter:     4 | Cost: 0.8531698 | Acc train: 0.7600000 | Acc validation: 0.7200000 | Acc test: 0.5166667 
Iter:     5 | Cost: 0.8985145 | Acc train: 0.6711111 | Acc validation: 0.7200000 | Acc test: 0.8583333 
Iter:     6 | Cost: 0.9580070 | Acc train: 0.5022222 | Acc validation: 0.6133333 | Acc test: 0.9166667 
Iter:     7 | Cost: 0.9119485 | Acc train: 0.6088889 | Acc validation: 0.6533333 | Acc test: 0.9166667 
Iter:     8 | Cost: 0.7929147 | Acc train: 0.7555556 | Acc validation: 0.8266667 | Acc test: 0.8833333 
Iter:     9 | Cost: 0.7388000 | Acc train: 0.8622222 | Acc validation: 0.8800000 | Acc test: 0.8166667 
Iter:    10 | Cost: 0.7742567 | Acc train: 0.8133333 | Acc valid

Load the test dataset and apply same transformations as we did with the train dataset.


In [27]:
print(
    "Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} | Acc test: {:0.7f} "
    "".format(it + 1, cost(weights, bias, features, Y), acc_train, acc_val, acc_test)
)


print('step: ', it)
print('bias: ', bias)    
print('weights:\n', weights)
np.save('/_jupyter/QC/QOSF-challenge-md-2022/z-temp/z-displacement-amplitude-b-001.npy', bias, allow_pickle=True)
np.save('/_jupyter/QC/QOSF-challenge-md-2022/z-temp/z-displacement-amplitude-w-001.npy', weights, allow_pickle=True)



Iter:    95 | Cost: 0.8986977 | Acc train: 0.6133333 | Acc validation: 0.6666667 | Acc test: 0.6500000 
step:  94
bias:  -0.6488172701721436
weights:
 [[[ 0.01126636 -0.01079932 -0.01147469]
  [ 0.02811479 -0.01214872  0.011485  ]
  [ 0.00949421  0.00087551 -0.01225436]
  [ 0.00844363 -0.01000215 -0.01544771]]

 [[ 0.0118803   0.00316943  0.00920859]
  [ 0.03475365  0.00355411 -0.01933477]
  [-0.01034243  0.00681595 -0.0080341 ]
  [-0.0068955  -0.00455533  0.00017479]]

 [[-0.00353994 -0.01374951 -0.00643618]
  [ 0.01047418 -0.00237985 -0.03747726]
  [-0.01104383  0.00052165 -0.00739563]
  [ 0.01543015 -0.01292857  0.00267051]]

 [[-0.00039283 -0.01168093  0.00523277]
  [ 0.03785403  0.01563076 -0.00530879]
  [ 0.02163236  0.01336528 -0.00369182]
  [-0.00239379  0.0109966   0.00655264]]

 [[ 0.00640132 -0.01616956 -0.00024326]
  [ 0.02896284  0.00546121 -0.01186338]
  [ 0.00910179  0.00317218  0.00786328]
  [-0.00466419 -0.00944446 -0.0041005 ]]

 [[-0.0001702   0.00379152  0.02259309]

In [28]:

acc_test = test_accuracy(weights, bias)
print('Accuracy on test data: {:0.4f}'.format(acc_test))


Accuracy on test data: 0.6500


In [29]:
# # We can plot the continuous output of the variational classifier for the first two dimensions of the data set.
# plt.figure()
# cm = plt.cm.RdBu

# # make data for decision regions
# xx, yy = np.meshgrid(np.linspace(0.0, 1.5, 300), np.linspace(0.0, 1.5, 300))
# X_grid = [np.array([x, y]) for x, y in zip(xx.flatten(), yy.flatten())]

# pd.DataFrame(xx).shape
# pd.DataFrame(yy).shape
# pd.DataFrame(X_grid).shape

In [30]:

# # # preprocess grid points like data inputs above
# # padding = 0.3 * np.ones((len(X_grid), 1))
# # X_grid = np.c_[np.c_[X_grid, padding], np.zeros((len(X_grid), 1))]  # pad each input

# # normalization = np.sqrt(np.sum(X_grid ** 2, -1))
# # X_grid = (X_grid.T / normalization).T  # normalize each input

# # features_grid = np.array(
# #     [get_angles(x) for x in X_grid]
# # )  # angles for state preparation are new features

# X_grid = X_norm
# features_grid = X_grid
# predictions_grid = [variational_classifier(weights, bias, f) for f in features_grid]
# # Z = np.reshape(predictions_grid, xx.shape)
# Z = predictions_grid 

# # plot decision regions
# cnt = plt.contourf(
#     xx, yy, Z, levels=np.arange(-1, 1.1, 0.1), cmap=cm, alpha=0.8, extend="both"
# )
# plt.contour(
#     xx, yy, Z, levels=[0.0], colors=("black",), linestyles=("--",), linewidths=(0.8,)
# )
# plt.colorbar(cnt, ticks=[-1, 0, 1])

# # plot data
# plt.scatter(
#     X_train[:, 0][Y_train == 1],
#     X_train[:, 1][Y_train == 1],
#     c="b",
#     marker="o",
#     edgecolors="k",
#     label="class 1 train",
# )
# plt.scatter(
#     X_val[:, 0][Y_val == 1],
#     X_val[:, 1][Y_val == 1],
#     c="b",
#     marker="^",
#     edgecolors="k",
#     label="class 1 validation",
# )
# # plt.scatter(
# #     X_train[:, 0][Y_train == -1],
# #     X_train[:, 1][Y_train == -1],
# #     c="r",
# #     marker="o",
# #     edgecolors="k",
# #     label="class -1 train",
# # )
# # plt.scatter(
# #     X_val[:, 0][Y_val == -1],
# #     X_val[:, 1][Y_val == -1],
# #     c="r",
# #     marker="^",
# #     edgecolors="k",
# #     label="class -1 validation",
# # )

# plt.legend()
# plt.show()